In [240]:
import pandas as pd
from datetime import datetime

In [241]:
# Number of file segment need to split
FILE_SEGMENT = 1

# 1. Read the dataset

In [242]:
data = pd.read_csv('./bus_running_times.csv', usecols=lambda column: column != 'Unnamed: 0')
data

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time_in_seconds,length,weather
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,69.0,0.6261,1
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,210.0,1.2808,1
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,496.0,2.1125,1
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,195.0,1.5513,1
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,97.0,0.8450,1
...,...,...,...,...,...,...,...,...,...,...
200657,25367.0,1377.0,2.0,30.0,2022-11-01,18:17:36,18:24:21,405.0,2.5600,1
200658,25367.0,1377.0,2.0,31.0,2022-11-01,18:24:21,18:25:36,75.0,0.4200,1
200659,25367.0,1377.0,2.0,32.0,2022-11-01,18:25:49,18:28:10,141.0,1.3000,1
200660,25367.0,1377.0,2.0,33.0,2022-11-01,18:28:10,18:31:25,195.0,1.2200,1


# 2. Understanding Dataset

In [243]:
def dataoveriew(data, message):
    print(f'{message}:\n')
    print('Number of rows: ', data.shape[0])
    print("\nNumber of features:", data.shape[1])
    print("\nData Features:")
    print(data.columns.tolist())
    print("\nMissing values:", data.isnull().sum().values.sum())
    print("\nUnique values:")
    print(data.nunique())

dataoveriew(data, 'Overview of the dataset')

Overview of the dataset:

Number of rows:  200662

Number of features: 10

Data Features:
['trip_id', 'deviceid', 'direction', 'segment', 'date', 'start_time', 'end_time', 'run_time_in_seconds', 'length', 'weather']

Missing values: 0

Unique values:
trip_id                14126
deviceid                  26
direction                  2
segment                   29
date                     272
start_time             51696
end_time               51683
run_time_in_seconds      999
length                    55
weather                    2
dtype: int64


## 2.1. Explore each Column

### trip_id

In [244]:
data['trip_id'].value_counts()

trip_id
1.0        15
9605.0     15
9603.0     15
9602.0     15
9598.0     15
           ..
20187.0     1
1746.0      1
24106.0     1
24760.0     1
6688.0      1
Name: count, Length: 14126, dtype: int64

Unique Trip id for every recorded bus trip

### deviceid

In [245]:
data['deviceid'].value_counts()

deviceid
274.0     15192
505.0     14470
279.0     14286
116.0     13522
123.0     13488
262.0     13277
128.0     10985
1143.0    10903
121.0     10874
1358.0    10503
250.0      8815
1408.0     8806
1377.0     8741
117.0      8670
264.0      7554
294.0      5510
1410.0     5130
513.0      5089
275.0      3292
1409.0     3227
1718.0     2855
1166.0     2496
1719.0     1486
284.0      1192
209.0       189
550.0       110
Name: count, dtype: int64

Total 26 devices are used to collect data on buses. Each device has been used on many trips. And this unique id for every bus running indicates a behavioural feature.

### Direction
Only has 2 values. direction 1 is from Kandy to Digana/Kadugannawa & direction 2 is vice versa

### Segment
Route segments which are in between stops named 1 to 15 for Kandy Digana direction while 21 to 34 for Digana to Kandy direction

### Date
Time window for the data collected: 01/10/2021 to 28/02/2022 & 01/07/2022 to 01/11/2022 (9 months)

### Start_time
Exiting time from the Bus stop OR start time for the segment

### End_time
Arriving time at the successive Bus Stop OR end time for the run in the segment

### Run_time_in_seconds
Running time in between the two stops (the unit of seconds)

### Length
Distance between two stops (the unit of kilometers)

# 3. Cleaning the dataset
Above section indicates dataset has many missing values (10116)
And the dataset has 203128 rows (so NAs account for about 4.9%)
I decide to remove all rows contains NA to predict more accurately

In [246]:
# Remove rows have missing values
data.dropna(inplace=True)

# 4. Splitting the dataset

In [247]:
df = data[data['segment']==FILE_SEGMENT]
df

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time_in_seconds,length,weather
14,1.0,262.0,1.0,15.0,2021-10-01,07:23:38,07:27:32,234.0,1.9667,1
29,8.0,274.0,1.0,15.0,2021-10-01,09:29:44,09:34:49,305.0,1.9667,1
53,15.0,262.0,1.0,15.0,2021-10-01,11:11:22,11:17:01,339.0,1.9667,1
68,21.0,274.0,1.0,15.0,2021-10-01,12:42:07,12:46:49,282.0,1.9667,1
78,22.0,123.0,1.0,15.0,2021-10-01,13:10:16,13:16:16,360.0,1.9667,1
...,...,...,...,...,...,...,...,...,...,...
171203,25364.0,1166.0,1.0,15.0,2022-11-01,17:30:06,17:35:54,348.0,1.9700,1
171216,25366.0,1358.0,1.0,15.0,2022-11-01,17:40:47,17:46:45,358.0,1.9700,1
171231,25368.0,284.0,1.0,15.0,2022-11-01,18:30:22,18:35:06,284.0,1.9700,1
171246,25369.0,513.0,1.0,15.0,2022-11-01,18:47:21,18:52:05,284.0,1.9700,1


### New Column: Date_in_week
Monday - Sunday: convert from colum `date`

In [248]:
# Extract Weekday, Saturday, Sunday form field Date
df['date'] = pd.to_datetime(df['date'])
df['date_in_week'] = df['date'].dt.day_name()
df

/var/folders/2p/0whgchk122l4cc9cl_4xk_ph0000gn/T/ipykernel_68611/200418102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/var/folders/2p/0whgchk122l4cc9cl_4xk_ph0000gn/T/ipykernel_68611/200418102.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_in_week'] = df['date'].dt.day_name()


,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time_in_seconds,length,weather,date_in_week
14,1.0,262.0,1.0,15.0,2021-10-01,07:23:38,07:27:32,234.0,1.9667,1,Friday
29,8.0,274.0,1.0,15.0,2021-10-01,09:29:44,09:34:49,305.0,1.9667,1,Friday
53,15.0,262.0,1.0,15.0,2021-10-01,11:11:22,11:17:01,339.0,1.9667,1,Friday
68,21.0,274.0,1.0,15.0,2021-10-01,12:42:07,12:46:49,282.0,1.9667,1,Friday
78,22.0,123.0,1.0,15.0,2021-10-01,13:10:16,13:16:16,360.0,1.9667,1,Friday
...,...,...,...,...,...,...,...,...,...,...,...
171203,25364.0,1166.0,1.0,15.0,2022-11-01,17:30:06,17:35:54,348.0,1.9700,1,Tuesday
171216,25366.0,1358.0,1.0,15.0,2022-11-01,17:40:47,17:46:45,358.0,1.9700,1,Tuesday
171231,25368.0,284.0,1.0,15.0,2022-11-01,18:30:22,18:35:06,284.0,1.9700,1,Tuesday
171246,25369.0,513.0,1.0,15.0,2022-11-01,18:47:21,18:52:05,284.0,1.9700,1,Tuesday


In [249]:
bins = list(range(4, 25))

In [250]:
labels = ['{}-{}'.format(i, i + 1) for i in bins[:-1]]

### Extract `hour` and `time_interval` column

In [251]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['hour'] = df['start_time'].dt.hour
df['time_interval'] = pd.cut(df['hour'], bins=bins, labels=labels, right=False)
df

/var/folders/2p/0whgchk122l4cc9cl_4xk_ph0000gn/T/ipykernel_68611/1510654388.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['start_time']).dt.hour
/var/folders/2p/0whgchk122l4cc9cl_4xk_ph0000gn/T/ipykernel_68611/1510654388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = pd.to_datetime(df['start_time']).dt.hour
/var/folders/2p/0whgchk122l4cc9cl_4xk_ph0000gn/T/ipykernel_68611/1510654388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time_in_seconds,length,weather,date_in_week,hour,time_interval
14,1.0,262.0,1.0,15.0,2021-10-01,07:23:38,07:27:32,234.0,1.9667,1,Friday,7,7-8
29,8.0,274.0,1.0,15.0,2021-10-01,09:29:44,09:34:49,305.0,1.9667,1,Friday,9,9-10
53,15.0,262.0,1.0,15.0,2021-10-01,11:11:22,11:17:01,339.0,1.9667,1,Friday,11,11-12
68,21.0,274.0,1.0,15.0,2021-10-01,12:42:07,12:46:49,282.0,1.9667,1,Friday,12,12-13
78,22.0,123.0,1.0,15.0,2021-10-01,13:10:16,13:16:16,360.0,1.9667,1,Friday,13,13-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171203,25364.0,1166.0,1.0,15.0,2022-11-01,17:30:06,17:35:54,348.0,1.9700,1,Tuesday,17,17-18
171216,25366.0,1358.0,1.0,15.0,2022-11-01,17:40:47,17:46:45,358.0,1.9700,1,Tuesday,17,17-18
171231,25368.0,284.0,1.0,15.0,2022-11-01,18:30:22,18:35:06,284.0,1.9700,1,Tuesday,18,18-19
171246,25369.0,513.0,1.0,15.0,2022-11-01,18:47:21,18:52:05,284.0,1.9700,1,Tuesday,18,18-19


## New Column: Part_Of_Day
 Early Morning: 3-7 (hour)
Morning: 7-10
 Noon: 10-14
Afternoon: 14-17
Evening: 17-20
Night: 20-24

In [252]:
def hour_to_part(hour):
    if (3<=hour<=12): return "morning" 
    if (12<hour<=18): return "afternoon"
    if (18<hour<=24): return "evening"
    # if (21<hour<=24): return "night"
    # if (17<hour<20): return "evening"
    # if (20<=hour<=24): return "night"
 
df['part_of_day'] = df['hour'].apply(hour_to_part)
df

/var/folders/2p/0whgchk122l4cc9cl_4xk_ph0000gn/T/ipykernel_68611/2365190794.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['part_of_day'] = df['hour'].apply(hour_to_part)


,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time_in_seconds,length,weather,date_in_week,hour,time_interval,part_of_day
14,1.0,262.0,1.0,15.0,2021-10-01,07:23:38,07:27:32,234.0,1.9667,1,Friday,7,7-8,morning
29,8.0,274.0,1.0,15.0,2021-10-01,09:29:44,09:34:49,305.0,1.9667,1,Friday,9,9-10,morning
53,15.0,262.0,1.0,15.0,2021-10-01,11:11:22,11:17:01,339.0,1.9667,1,Friday,11,11-12,morning
68,21.0,274.0,1.0,15.0,2021-10-01,12:42:07,12:46:49,282.0,1.9667,1,Friday,12,12-13,morning
78,22.0,123.0,1.0,15.0,2021-10-01,13:10:16,13:16:16,360.0,1.9667,1,Friday,13,13-14,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171203,25364.0,1166.0,1.0,15.0,2022-11-01,17:30:06,17:35:54,348.0,1.9700,1,Tuesday,17,17-18,afternoon
171216,25366.0,1358.0,1.0,15.0,2022-11-01,17:40:47,17:46:45,358.0,1.9700,1,Tuesday,17,17-18,afternoon
171231,25368.0,284.0,1.0,15.0,2022-11-01,18:30:22,18:35:06,284.0,1.9700,1,Tuesday,18,18-19,afternoon
171246,25369.0,513.0,1.0,15.0,2022-11-01,18:47:21,18:52:05,284.0,1.9700,1,Tuesday,18,18-19,afternoon


In [253]:
df.to_csv(f'./segment{FILE_SEGMENT}.csv', header=True)